# End-to-end example: Taking the train to the slopes

![bourg](img/bourg.png)

(Photo: the author)

Geospatial analysis often includes combining multiple data sources. In this example, we will only use OpenStreetMap (OSM), but as you'll see one of the datasets is a LineString type and another is a Polygon type.

::: {.callout-note}

Why don't we use Overture Maps, you might ask, as that's already available in a cloud-native format? Very good question. Unfortunately, Overture Maps doesn't (yet) contain all datasets that you can find in OSM, so while roads and buildings are included, for example transit isn't (as of writing).

:::

Let's say we'd like to go skiing but would like to avoid the hassle of driving or flying, Are there slopes accessible by train? It turns out, yes -- let's find them.

## Fetching OSM

See also [Importing other formats](../other_formats/import.ipynb) on how to use DuckDB Spatial to read OSM and other file types.

In [0]:
%pip install duckdb lonboard shapely --quiet

import duckdb

duckdb.sql("install spatial; load spatial;")

In [0]:
CATALOG = "workspace"
SCHEMA = "default"
VOLUME = "default"

CONTINENT = "europe"
COUNTRY = "france"
GEOFABRIK_URL = f"https://download.geofabrik.de/{CONTINENT}/{COUNTRY}-latest.osm.pbf"

spark.sql(f"use {CATALOG}.{SCHEMA}")
spark.sql(f"create volume if not exists {CATALOG}.{SCHEMA}.{VOLUME}")

In [0]:
file_name = GEOFABRIK_URL.split("/")[-1]
file_basename = file_name.rsplit(".")[0]
volume_file_path = f"/Volumes/{CATALOG}/{SCHEMA}/{VOLUME}/{file_name}"
volume_parquet_path = f"/Volumes/{CATALOG}/{SCHEMA}/{VOLUME}/{file_basename}.parquet"
tablename = f"planet_osm_{COUNTRY}"

In [0]:
!curl -o {volume_file_path} {GEOFABRIK_URL}

In [0]:
duckdb.sql(
    f"""
copy (
    select
        *
    from
        '{volume_file_path}'
) to '{volume_parquet_path}'
(format parquet)
;
"""
)

In [0]:
spark.read.parquet(volume_parquet_path).write.option("clusteredBy", "id").saveAsTable(
    f"{CATALOG}.{SCHEMA}.{tablename}"
)

## Transform into a table with GEOMETRY with Databricks Spatial SQL

See also [Databricks Spatial SQL ST functions](../stfunctions/native.ipynb), and [Delta Lake tables with GEOMETRY](../delta/GEOMETRY.ipynb).

In [0]:
%sql
create or replace table skiresorts as
with wintersports as (
  select
    id as wintersports_id,
    tags.name,
    posexplode(refs) as (pos, id)
  from
    planet_osm_france
  where
    kind = "way"
    and tags.landuse = "winter_sports"
)
select
  wintersports_id,
  name,
  st_makepolygon(
    st_makeline(
      transform(sort_array(array_agg(struct(pos, lon, lat))), x -> st_point(x.lon, x.lat, 4326))
    )
  ) geometry
from
  wintersports join planet_osm_france p using (id)
group by
  all

::: {.callout-note}

As of 2025-08-20, GEOMETRY/GEOGRAPHY columns cannot be returned in a notebook yet, therefore instead of `select *`, we temporarily use the pattern, for a column `g` of type GEOMETRY/GEOGRAPHY:

```sql
select
  * except (g),
  st_asewkt(g) as ewkt_g
```

:::

In [0]:
%sql
select
  * except (geometry),
  st_asewkt(geometry) geometry
from
  skiresorts
-- Returns:
-- wintersports_id	name	geometry
-- 23079840	Isola 2000	SRID=4326;POLYGON((7.1524774 44.204359200000006,...))
-- 27163365	Estación de Esquí Aramón Formigal	SRID=4326;POLYGON((-0.41821430000000004 42.8009106,...))
-- ...

## Visualize with Lonboard

See also [Visualize with Lonboard](../viz/lonboard.ipynb).

In [0]:
from lonboard import viz
from pyspark.sql import functions as F

dfa = (
    spark.table("skiresorts")
    .withColumn("geometry", F.expr("st_asbinary(geometry)"))
    .toArrow()
)
viz(duckdb.sql("select name, st_geomfromwkb(geometry) geometry from dfa")).as_html()

![wintersport](img/wintersport.png)

## Load the train network

We will use similar techniques as above to load the railway network as well.

In [0]:
%sql
with t1 as (
  select
    tags.name,
    id as trainroute_id,
    posexplode(refs) as (pos, id)
  from
    planet_osm_france
  where
    tags.type = 'route'
    and tags.route = 'train'
    and id = 2274158
),
t2 as (
  select
    t1.name,
    p.id route_id,
    p.* --,
  -- posexplode(arrays_zip(p.refs, p.ref_roles, p.ref_types)) as (pos, ref),
  -- ref["refs"] id,
  -- ref["ref_roles"] role,
  -- ref["ref_types"] type
  from
    t1 join planet_osm_france p using (id)
  where
    p.tags.railway = 'rail'
)
select
  *
from
  t2

In [0]:
%sql
create or replace table train_routes as
with t1 as (
  select
    tags.name,
    id as trainroute_id,
    posexplode(refs) as (pos, id)
  from
    planet_osm_france
  where
    tags.type = 'route'
    and tags.route = 'train'
),
t2 as (
  select
    t1.name,
    p.id route_id,
    posexplode(refs) as (pos, id)
  from
    t1 join planet_osm_france p using (id)
  where
    p.tags.railway = 'rail'
)
select
  t2.name,
  --subname,
  route_id,
  --subroute_id,
  st_makeline(
    transform(sort_array(array_agg(struct(pos, lon, lat))), x -> st_point(x.lon, x.lat, 4326))
  ) geometry
from
  t2 join planet_osm_france p using (id)
group by
  all

In [0]:
spark.table("train_routes").withColumn(
    "geometry", F.expr("st_asewkt(geometry)")
).display()
# Returns:

# [lots of LINESTRING's]

Let's try to visualize the same way as we did for the ski _domaines_ (spoiler alert: it might not work):

In [0]:
dfa = (
    spark.table("train_routes")
    .withColumn("geometry", F.expr("st_asbinary(geometry)"))
    .toArrow()
)
viz(duckdb.sql("select name, st_geomfromwkb(geometry) geometry from dfa")).as_html()
# Returns:
# Command result size exceeds limit: Exceeded 20971520 bytes (current = 20971797)

## Visualize with PMTiles

The above visualization probably failed, due to the dataset being too large for the widget in Databricks. While for medium sized datasets there's another workaround by saving the Lonboard output to a seperate HTML file via `to_html()` (detailed [here](../viz/lonboard.ipynb)), let's instead use PMTiles instead that can work also for very large datasets.

See also the `tippecanoe` example [here](../other_formats/export.ipynb), which we will follow now via an intermediate Parquet and FlatGeobuf file.

In [0]:
# Write to parquet
spark.table("train_routes").write.mode("overwrite").parquet(
    f"/Volumes/{CATALOG}/{SCHEMA}/{VOLUME}/geometries.parquet"
)

In [0]:
# Write FlatGeobuf
query = f"""
copy (
select 
    * replace (st_geomfromwkb(geometry) as geometry)
from
    read_parquet('/Volumes/{CATALOG}/{SCHEMA}/{VOLUME}/geometries.parquet/part-*.parquet')
) to '/Volumes/{CATALOG}/{SCHEMA}/{VOLUME}/geometries.fgb'
(FORMAT GDAL, DRIVER flatgeobuf, LAYER_CREATION_OPTIONS 'TEMPORARY_DIR=/tmp/')"""
duckdb.sql(query)

In [0]:
%sh
# Install tippecanoe

# ~3.5 min on high memory serverless https://docs.databricks.com/aws/en/compute/serverless/dependencies#high-memory
git clone https://github.com/felt/tippecanoe.git
cd tippecanoe
make -j
make install PREFIX=$HOME/.local
rm -r tippecanoe

In [0]:
import os

HOME = os.environ["HOME"]

# see https://github.com/felt/tippecanoe/blob/main/README.md#try-this-first and e.g.
# https://github.com/OvertureMaps/overture-tiles/blob/main/scripts/2024-07-22/places.sh
# for possible options
!{HOME}/.local/bin/tippecanoe -zg -rg -o /tmp/geometries.pmtiles  --simplification=10 --drop-smallest-as-needed --drop-densest-as-needed --extend-zooms-if-still-dropping --maximum-tile-bytes=2500000 --progress-interval=10 -l geometries --force /Volumes/{CATALOG}/{SCHEMA}/{VOLUME}/geometries.fgb
# NOTE: this mv will emit an error related to updating metadata ("mv: preserving
# permissions for ‘[...]’: Operation not permitted"), this can be ignored.
!mv /tmp/geometries.pmtiles /Volumes/{CATALOG}/{SCHEMA}/{VOLUME}/geometries.pmtiles

And, that's it! You can visualize the pmtiles file by downloading and uploading to https://pmtiles.io , or much better, by using a PMTiles viewer via Databricks Apps, an example implementation is [here](../apps/pmtiles-viewer/) and your result would look like this:

![railnetwork](img/railnetwork.png)

## Spatial join


To actually answer our question of which ski resorts are closest to a train line -- for now, I leave that as an exercise to the reader as spatial joins are well covered in other tutorials -- one hint is that most probably you'd need to [`st_transform`](https://docs.databricks.com/aws/en/sql/language-manual/functions/st_transform) to a local coordinate system, because while there's an [`st_distancespheroid`](https://docs.databricks.com/aws/en/sql/language-manual/functions/st_distancespheroid) function, as of writing it only works with points.